In [257]:
import numpy as np
from tqdm import trange
import time

In [333]:
def lp(trX, X, Z, sigma_X = 1, sigma_A = 1):
    N, D = X.shape
    
    K = Z.shape[1]
    
    u, s, v = np.linalg.svd(Z,full_matrices=False)
    det = np.sum(np.log(s**2 + sigma_X**2 / sigma_A**2))
    l = s**2 / (s**2 + sigma_X**2 / sigma_A**2)

    uTX = u.T @ X
    uX = np.sum(uTX ** 2, axis = 1)

    res = - N * D / 2 * np.log(2 * np.pi)
    res -= (N - K) * D * np.log(sigma_X)
    res -= K * D * np.log(sigma_A)
    res -= D / 2 * det
    res -= 1 / (2 * sigma_X**2) * (trX - sum(l * uX))
    return res

In [334]:
def lp_optimization(X, Z, C, i, sigma_X = 1, sigma_A = 1):
    e = np.zeros(Z.shape[0])
    e[i] = 1
    trX = np.trace(X.T @ X)
    base = lp(trX, X, Z, sigma_X, sigma_A)
    diff = [0] * C
    for c in range(C):
        Z = np.c_[Z, e]
        diff[c] = lp(trX, X, Z, sigma_X, sigma_A) - base
    return diff

In [350]:
def lp_recursive(X, Z, C, i, sigma_X = 1, sigma_A = 1):
    D = X.shape[1]
    e = np.zeros(Z.shape[0])
    e[i] = 1
    U, S, Vh = np.linalg.svd(Z, full_matrices = False)
    d = S ** 2 / (S ** 2 + sigma_X**2/sigma_A**2)
    coef = d * U[i,]
    gamma_i = U @ coef
    cur_diff = 0
    diff = [0] * C
    for c in range(C):
        mu = 1 + sigma_X**2/sigma_A**2 - gamma_i[i]
        t = 1/mu * np.sum((X.T @ gamma_i - X[i])**2)
        cur_diff += D * np.log(sigma_X / sigma_A)
        cur_diff -= D / 2 * np.log(mu) 
        cur_diff += t/(2 * sigma_X**2)
        gamma_i += 1/mu * (gamma_i[i] - 1) * (gamma_i - e)
        diff[c] = cur_diff
    return diff

In [336]:
def compare(N, D, K, C, i, sigma_X = 1, sigma_A = 1):
    
    X = np.random.normal(0, 1, (N, D))
    Z = np.random.randint(0, 2, (N, K))
    time_1 = time.time()
    opt = lp_optimization(X, Z, C, i, sigma_X, sigma_A)
    time_opt = time.time() - time_1
    time_2 = time.time()
    rec = lp_recursive(X, Z, C, i, sigma_X, sigma_A)
    time_rec = time.time() - time_2
    return np.array(opt) - np.array(rec), time_opt, time_rec

In [349]:
def multicompare(N, D, K, C, T):
    res = np.zeros((3, T))
    for t in trange(T):
        i = np.random.randint(0, N)
        sigma_X = np.random.rand() + 1
        sigma_A = np.random.rand() + 1
        err, topt, trec = compare(N, D, K, C, i, sigma_X, sigma_A)
        res[0, t] = np.max(err)
        res[1, t] = topt
        res[2, t] = trec
    return np.mean(res[0]), np.mean(res[1]), np.max(res[1]), np.mean(res[2]), np.max(res[2])

In [346]:
multicompare(N = 5000, D = 1000, K = 250, C = 15, T = 1)

(9.64064383879304e-10,
 1.647707223892212,
 1.647707223892212,
 0.08820128440856934,
 0.08820128440856934)